In [19]:
import torch
from transformers import MarianMTModel, MarianTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
dataset_name = "Iker/Document-Translation-en-es"

summary_model_name = "t5-small"
translation_model_name = "Helsinki-NLP/opus-mt-en-es"
revision = "main"

device = 0 if torch.cuda.is_available() else -1

## Base summariser

In [21]:
model = AutoModelForSeq2SeqLM.from_pretrained(summary_model_name)
tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
text = """
The Hugging Face Transformers library provides state-of-the-art general-purpose architectures
for natural language understanding (NLU) and natural language generation (NLG). These architectures
include BERT, GPT, GPT-2, BART, and T5, which can be applied to text classification, summarization, translation, and more.
"""

inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

the Hugging Face Transformers library provides state-of-the-art architectures for natural language understanding (NLU) and natural language generation (NLG). these architectures include BERT, GPT, GPT-2, and BART.


## Baseline translator

In [14]:
tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
model = MarianMTModel.from_pretrained(translation_model_name)
english_text = "Let's learn AI using Python"
input_ids = tokenizer.encode(english_text, return_tensors="pt")
translated_ids = model.generate(input_ids)

translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

print("Translated Text (Spanish):", translated_text)

/Users/marek/dev/llm_venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/Users/marek/dev/llm_venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Translated Text (Spanish): Vamos a aprender IA usando Python


## Load dataset

In [22]:
from datasets import load_dataset

dataset = load_dataset(dataset_name)
train_dataset = dataset["train"].shuffle(seed=42).select(range(100))  # Using a subset for quick fine-tuning
test_dataset = dataset["test"].shuffle(seed=42).select(range(100))

Generating train split: 100%|██████████| 10533/10533 [00:00<00:00, 119201.57 examples/s]


KeyError: 'test'

## Load tokenizer

In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    # print(examples["text"][0])
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
